In [1]:
class treeNode():
    def __init__(self,feat,val,right,left):
        featureToSsplitOn = feat
        valueOfSplit =val
        leftBranch = left

In [133]:
from numpy import linalg,mat,corrcoef,eye,nonzero
import numpy as np
import matplotlib.pyplot as plt

def loadDataSet(fileName):      #general function to parse tab -delimited floats
    dataMat = []                #assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float,curLine)) #map all elements to float()
        dataMat.append(fltLine)
    return dataMat

In [134]:
ll

total 96
-rw-rw-r-- 1 xu  4235 2月  15  2011 bikeSpeedVsIq_test.txt
-rw-rw-r-- 1 xu  4220 2月  15  2011 bikeSpeedVsIq_train.txt
-rw-rw-r-- 1 xu  3846 2月   8  2011 ex00.txt
-rw-rw-r-- 1 xu  5621 2月   8  2011 ex0.txt
-rw-rw-r-- 1 xu  4064 2月   9  2011 ex2test.txt
-rw-rw-r-- 1 xu  4069 2月   9  2011 ex2.txt
-rw-rw-r-- 1 xu  3831 2月  10  2011 exp2.txt
-rw-rw-r-- 1 xu  4011 2月  14  2011 expTest.txt
-rw-rw-r-- 1 xu  3998 2月   9  2011 exp.txt
-rw-rw-r-- 1 xu  5712 2月  15  2011 regTrees.py
-rw-rw-r-- 1 xu  3879 2月  14  2011 sine.txt
-rw-rw-r-- 1 xu  2383 2月  15  2011 treeExplore.py
-rw-rw-r-- 1 xu 29906 5月  21 21:46 树回归.ipynb


In [4]:
def binSplitDataSet(dataSet, feature, value):
    mat0 = dataSet[nonzero(dataSet[:,feature]>value)[0],:][0]
    mat1 = dataSet[nonzero(dataSet[:,feature]<=value)[0],:][0]
    return mat0, mat1


In [94]:
def createTree(dataSet, leafType=regLeaf, errType = regErr, ops=(1,4)):
    feat ,val = chooseBestSplit(dataSet,leafType,errType,ops)
    if feat == None:return val
    retTree = {}
    retTree['spInd']=feat
    retTree['spVal']=val
    lSet, rSet = binSplitDataSet(dataSet,fet,val)
    retTree['left'] = createTree(lSet,leafType,errType,ops)
    retTree['right'] = createTree(rSet,leafType,errType,ops)
    return retTree

In [5]:
testMat = mat(eye(4))

In [6]:
testMat

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]])

In [7]:
binSplitDataSet(testMat,1,0.5)

(matrix([[ 0.,  1.,  0.,  0.]]), matrix([[ 1.,  0.,  0.,  0.]]))

In [8]:
testMat[nonzero(testMat)]!=0

matrix([[ True,  True,  True,  True]], dtype=bool)

In [64]:
testMat.tolist()[:,-1]

TypeError: list indices must be integers or slices, not tuple

In [10]:
from pandas import DataFrame

In [144]:
from numpy import mean,var,square,shape,inf

# chooseBestSplit

In [14]:
def regLeaf(dataSet):
    return mean(dataSet[:,-1])

In [15]:
def regErr(dataSet):
    return var(dataSet[:-1])*shape(dataSet)[0]

In [218]:
def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr,ops=(1,4)):
    tolS = ops[0];tolN = ops[1]
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1:
        print('1111111')
        return None, leafType(dataSet)
    m,n=shape(dataSet)
    print(m,n)
    S = errType(dataSet)
    bestS = inf;bestIndex = 0; bestValue = 0
    for featIndex in range(n-1):
        for splitVal in set(dataSet[:,featIndex].T.tolist()[0]):
            print('ininin')
            mat0,mat1 = binSplitDataSet(dataSet,featIndex,splitVal)
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
                continue
            newS = errType(mat0) +errType(mat1)
            if newS < bestS:
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    if (S - bestS) < tolS:
        print('2222222222')
        return None, leafType(dataSet)
    mat0, mat1 = binSplitDataSet(dataSet,bestIndex,bestValue)
    if (shape(mat0)[0]<tolN) or (shape(mat1)[0]<tolN):
        print('33333333333333')
        return None,leafType(dataSet)
    return bestIndex, bestValue

In [224]:
def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):
    tolS = ops[0]; tolN = ops[1]
    #if all the target variables are the same value: quit and return value
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1: #exit cond 1
        return None, leafType(dataSet)
    m,n = shape(dataSet)
    #the choice of the best feature is driven by Reduction in RSS error from mean
    S = errType(dataSet)
    bestS = inf; bestIndex = 0; bestValue = 0
    for featIndex in range(n-1):
        for splitVal in set(dataSet[:,featIndex].T.tolist()[0]):
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN): continue
            newS = errType(mat0) + errType(mat1)
            if newS < bestS: 
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    #if the decrease (S-bestS) is less than a threshold don't do the split
    if (S - bestS) < tolS: 
        return None, leafType(dataSet) #exit cond 2
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):  #exit cond 3
        return None, leafType(dataSet)
    return bestIndex,bestValue#returns the best feature to split on
                              #and the value used for that split


#  测试

In [225]:
myDat = loadDataSet('ex00.txt')

In [226]:
myMat = mat(myDat)

In [227]:
createTree(myMat)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [211]:
set(myMat[:,0].T.tolist()[0])

{0.000234,
 0.007044,
 0.008307,
 0.009849,
 0.014083,
 0.036098,
 0.036492,
 0.048014,
 0.065615,
 0.066172,
 0.071769,
 0.072243,
 0.074795,
 0.081931,
 0.084248,
 0.091358,
 0.096994,
 0.098016,
 0.099142,
 0.101149,
 0.118156,
 0.129061,
 0.130052,
 0.130962,
 0.132543,
 0.1333,
 0.145809,
 0.146366,
 0.148049,
 0.152324,
 0.180506,
 0.18351,
 0.188218,
 0.193641,
 0.196005,
 0.202054,
 0.203693,
 0.210334,
 0.233115,
 0.23807,
 0.242204,
 0.243909,
 0.250744,
 0.252649,
 0.268857,
 0.269681,
 0.273147,
 0.27394,
 0.278661,
 0.280738,
 0.295511,
 0.302001,
 0.302217,
 0.30554,
 0.306964,
 0.314924,
 0.323181,
 0.325412,
 0.332982,
 0.3371,
 0.339563,
 0.343479,
 0.343554,
 0.344102,
 0.346986,
 0.347837,
 0.348013,
 0.355688,
 0.360323,
 0.362314,
 0.377201,
 0.377597,
 0.378595,
 0.401152,
 0.406649,
 0.41023,
 0.429664,
 0.438367,
 0.448656,
 0.44928,
 0.458121,
 0.465625,
 0.475976,
 0.48813,
 0.50794,
 0.514563,
 0.517921,
 0.518735,
 0.520044,
 0.520207,
 0.527505,
 0.529491,


In [109]:
myMat[:,-1].T.to_list()

AttributeError: 'matrix' object has no attribute 'to_list'